# Disease simulation on temporal network data

This simulation was originally made to examine the outcomes of disease outbreaks under various different conditions regarding the type and severity of people's behavior change in responses to the symptoms of an infectious disease. However, by choosing the parameter values appropriately, the simulation can also be used to test situations where there is no behavioral reaction. The code described in this notebook is can also be found in the same repository, temporal_simulation.py is the main simulation code and example.py is is a working example of how to implement it (the same example shown here). 

## Data

Lets choose a dataset and read it in. The data sets are 'conference', 'hospital', 'looped_primary_school', and 'synthetic_temporal_data', all taken from sociopatterns.org. In these studies, participants wore radiofrequency identification (RFID) sensors that detect face-to-face proximity of other participants within $1-1.5$ meters in $20$-second intervals. Each data-set lists the identities of the people in contact, as well as the $20$-second interval of detection. These data have been formatted to appear in a table with the start and end times of the interaction listed instead. To exclude contacts detected while participants momentarily walked past one another, only contacts that are detected in at least two consecutive intervals are considered interactions. 

The three data-sets used were: (a) a conference in which $110$ participants were recorded over $3$ days, (b) a hospital ward in which $74$ participants were recorded over $4$ days, and (c) a primary school in which $242$ participants were recorded over $2$ days. We looped this data to produce a dataset spanning six weeks. We used the first day to represent Monday, Wednesday and Friday and the second day to represent Tuesday and Thursday. We then added 2 days of inactivity to replicate a typical school week and weekend.

In [10]:
import pandas as pd
data='looped_primary_school'
original_df=pd.read_csv('data/'+data+'.txt', sep='\t', header=None, names=['ID1','ID2','start_time','end_time'])
# read it again with edge directions reversed (so that the disease can go in both directions)
reverse_df=pd.read_csv('data/'+data+'.txt', sep='\t', header=None, names=['ID2','ID1','start_time','end_time'])    
#put them together
df=pd.concat([original_df,reverse_df]) 

print(df.head())

    ID1   ID2  end_time  start_time
0  1538  1539     56580       56520
1  1538  1546     35340       35180
2  1538  1546     37720       37480
3  1538  1546     55960       55900
4  1538  1546     56060       56000


To make the simulation both simple and fast it is beneficial to keep the data in the following format:

In [15]:
ID_list=list(set(df['ID1']))     
contacts={}
for node in ID_list:
    node_df=df[(df['ID1']==node)]
    names=node_df['ID2'].tolist()
    start_times=node_df['start_time'].tolist()
    end_times=node_df['end_time'].tolist()
    contacts[node]=[]
    for i in range(len(names)):
        contacts[node].append([names[i],start_times[i],end_times[i],0])

In addition to the data we need to choose the parameters that describe the disease model. Specifically these are:

## Transmissibility $\beta$

$\beta$ is the probability that transmission occurs during any one second of contact between an infectious individual and a susceptible individual.

## Latent period mode, $\hat{\Delta}_{E}$, and  dispersion, $\sigma_{g}^{(E)}$

The duration of the latent period may vary between individuals depending on their age, gender, or other characteristics. While the latent period and the incubation period are not the same, we assume that the biological factors determining their length to be similar, and thus we assume that the distribution of latent periods is log-Normal \cite{sartwell1950distribution}. In the simulation, the latent duration for each infected individual is drawn from a log-Normal distribution with mode $\hat{\Delta}_{E}$ and dispersion factor $\sigma_{g}^{(E)}$ (the geometric standard deviation of the distribution). We use $\sigma=\sigma_{g}^{(E)}$ and $\mu= \sigma^{2}+\log(\hat{\Delta}_{E})$ to get the standard parameters for the log-Normal distribution.

##  Infections period mode, $\hat{\Delta}_{I}$, and perseverance, $1/k_{I}$

Once infected, the behavioral response of individuals may vary; some might leave the system (or take other measures to prevent infection) immediately, whereas some may remain a risk to others for a more prolonged duration. In the simulation, the duration of the infectious period of each individual is randomly selected from a gamma distribution with a mode of $\hat{\Delta}_{I}$ hours. We define perseverance as $1/k_{I}$ where $k_{I}$ is the shape parameter of the gamma distribution. By choosing the scale parameter of the Gamma distribution to be $\theta=\hat{\Delta}_{I}/(k_{I}-1)$ we ensure that the mode does not change while increasing the perseverance fattens the the tail of the distribution.

## Asymptomatic proportion $a$

Some members of the population may show no signs of infection (up to $28\%$ reported for influenza and $32\%$ for rhinovirus), or might just ignore them completely, in which case their behavior does not change. At the beginning of the simulation, a random sample of the population are chosen to be asymptomatic. These individuals, who make up a fraction $a$ of the total population, have an infectious period of $24$ hours (this is hard-coded in the simulation).

We keep all of these parameters in a dictionary:

In [20]:
params={'beta':0.001,
            'l_mode':22, 
            'l_dispersion':1.1,
            'i_mode':2,
            'i_shape':5,
            'asymptomatic_proportion':0.0,
            }

Finally we need to have a seed individual from which the outbreak begins, and we need to choose a time for which the infectious period of the seed individual begins. Here we just choose a random individual and let their infectious period begin the moment they start interacting:

In [23]:
import numpy.random as rdm
seed=ID_list[int(len(ID_list)*rdm.random())]
time=min([contact[1] for contact in contacts_dict[seed]])

The simulation procedure runs as follows. Arrows show the order in which operations are to be completed. Blue boxes represent conditional rules of the algorithm; green ticks indicate the next operation if the answer is yes, red crosses if the answer is no. To make it easier to follow, the diagram below does not show how asymptomatic individuals are included in the simulation. 

<img src="Figs/Disease_simulation_flow_chart.png" alt="Drawing" style="width:600px;"/>

This is what it looks like in Python code:

In [24]:
import numpy as np
import random

susceptible_nodes=list(set(contacts.keys()))
N=len(susceptible_nodes)

time_of_infection={}

source_of_infection={seed:None}
#make a proportion of the nodes asymptomatic
asymptomatic_nodes=random.sample(susceptible_nodes,int(N*params['asymptomatic_proportion']))

#infection_tree is the tree network of infections that the function returns
infection_tree=[]    
#keep a list of infected nodes (at the beginning it contains only 'node')    
infections=[[seed,time]]

#get the parameters for the lognormal distribution
sigma=np.log(params['l_dispersion'])
mu=(sigma**2)+np.log(params['l_mode'])

while len(infections) > 0:

    #get earliest infection event on the list
    next_infection=min(infections,key=lambda x: x[1])        
    #remove the chosen infection
    infections.remove(next_infection)

    infectious_node=next_infection[0]
    #add it to the immune list
    susceptible_nodes.remove(infectious_node)     
    #update the time
    time=next_infection[1]

    #add to the tree
    infection_tree.append(next_infection+[source_of_infection[infectious_node]])

    #randomly select a latent duration 
    if infectious_node==seed:
        latent_duration=0
    else:
        latent_duration=int(60*60*np.random.lognormal(mu,sigma))

    #randomly select an infectious duration for each type
    infectious_duration={}
    for i in range(2):
        if infectious_node in asymptomatic_nodes:
            infectious_duration=24*60*60
        else:
            infectious_duration=int(60*60*np.random.gamma(params['i_shape'], scale=params['i_mode']/(params['i_shape']-1)))

    #contact_list=contacts[infectious_node].copy()#
    contact_list=[contact for contact in contacts[infectious_node] if contact[1]<time+latent_duration+infectious_duration and contact[2]>time+latent_duration]
    #loop over all the potentially infectious interactions
    while len(contact_list) > 0:
        contact=contact_list.pop()

        name=contact[0]
        contact_start=contact[1]
        contact_end=contact[2]

        #exposure starts either at the start of infectious period or the start of interaction, whichever is later
        exposure_start=max(time+latent_duration,contact_start)
        #exposure endes at the end of infectious period or the end of interaction, whichever is earlier
        exposure_end=min(time+latent_duration+infectious_duration,contact_end)                          

        #select a random time for the infection to occur
        r=random.random()
        #this is equivalent to an attempt at transmission occuring each second (different beta depending on the location)           
        l=l=-np.log(1-params['beta'])              
        #l=-np.log(1-b)                              
        infection_time=exposure_start+int(-(1/l)*np.log(1-r))             

        #check that the interaction is with a susceptible node, that it is not a sick day at work                                  
        if infection_time<exposure_end and name in susceptible_nodes:
            #if this is the case then a potential infection occurs!

            #if 'name' is already in the list of infections then check to see which happened first
            if name in [i[0] for i in infections]:
                if infection_time<time_of_infection[name]:
                    #remove the later infection time
                    infections.remove([name,time_of_infection[name]])
                    #add the earlier infection time
                    infections.append([name,infection_time])
                    #keep track of where it came from
                    source_of_infection[name]=infectious_node
                    #update the list
                    time_of_infection[name]=infection_time
            #if this is the first time 'name' has been infected then add it to the list
            else:
                #keep track of the time of the infection
                time_of_infection[name]=infection_time
                #keep track of who it came from
                source_of_infection[name]=infectious_node
                #update the list
                infections.append([name,infection_time])

Now we are interested in knowing who infected whom in the simulation. This is the infection tree. We thus have:

In [25]:
for t in infection_tree:
    print(t[0],'infected by',t[2],'at time',t[1])

1628 infected by None at time 31540
1613 infected by 1628 at time 35511
1625 infected by 1628 at time 35840
1911 infected by 1625 at time 122298
1647 infected by 1613 at time 126515
1902 infected by 1613 at time 129045
1855 infected by 1613 at time 129870
1708 infected by 1613 at time 131596
1700 infected by 1613 at time 131788
1780 infected by 1613 at time 132346
1738 infected by 1780 at time 204980
1737 infected by 1780 at time 208200
1872 infected by 1855 at time 208254
1822 infected by 1780 at time 215721
1552 infected by 1700 at time 216603
1579 infected by 1700 at time 220235
1743 infected by 1708 at time 224300
1564 infected by 1822 at time 293217
1555 infected by 1738 at time 294963
1731 infected by 1822 at time 295267
1887 infected by 1743 at time 311128
1558 infected by 1564 at time 377646
1722 infected by 1555 at time 384403
